In [1]:
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider, TextInput
from bokeh.plotting import figure, output_file, show, ColumnDataSource

def cost_curve(x,premium,contribution,deductible,coinsurance,maxOutOfPocket):
    eff_premium = premium - contribution
    post_deduct = (coinsurance * (x - deductible)) + deductible
    y = np.fmin(np.fmin(x, post_deduct), maxOutOfPocket) + eff_premium
    return y

def plan_UI(x=np.linspace(0.0, 10000, 100),premium=100,contribution=50,deductible=2000,coinsurance=0.5,maxOutOfPocket=4000,name="Plan"):
    y = cost_curve(x,premium,contribution,deductible,coinsurance,maxOutOfPocket)
    c = [name for i in range(len(x))]
    plan = {
        "label": TextInput(value=name, title="Plan Name"),
        "source": ColumnDataSource(data=dict(x=x, y=y, category=c), name=name),
        "premium": Slider(start=0, end=2000, value=premium, step=.1, title="Premium"),
        "contribution": Slider(start=0, end=1000, value=contribution, step=10, title="Company Contribution"),
        "deductible": Slider(start=0, end=6000, value=deductible, step=10, title="Deductible"),
        "coinsurance": Slider(start=0, end=1, value=coinsurance, step=.01, title="Coinsurance Rate (paid by you)"),
        "maxOutOfPocket": Slider(start=0, end=8000, value=maxOutOfPocket, step=10, title="Maximum Out Of Pocket")
    }
    return plan

hover_tooltips = [
    ("Plan", "@category"),
    ("Spent", "@x{($ 0.00 a)}"),
    ("Incurred", "@y{($ 0.00 a)}"),
]

callback_code = """
    const data = source.data;
    const P = premium.value - contribution.value;
    const D = deductible.value;
    const R = coinsurance.value;
    const M = maxOutOfPocket.value;
    const x = data['x']
    const y = data['y']
    const c = data['category']
    const L = label.value;
    for (var i = 0; i < x.length; i++) {
        const post_deduct = (R*(x[i] - D)) + D;
        y[i] = Math.min(Math.min(x[i],post_deduct),M) + P;
        c[i] = L;
    }
    source.change.emit();
"""

In [2]:
x = np.linspace(0.0, 10000, 100)

plan1 = plan_UI(x, 618.96, 300, 3000, 0.65, 7000,"HSA")
plan2 = plan_UI(x, 1200.04, 400, 2250, 0.65, 4700,"HRA")

plot = figure(x_range=(0,10000), y_range=(0,10000), plot_width=400, plot_height=400, tools="pan,hover,crosshair,reset", tooltips=hover_tooltips)
plot.line(x, x, line_dash=(4,4), line_color="black")
plot.xaxis.axis_label = 'Out-of-Pocket Expenses'
plot.yaxis.axis_label = 'Actual Incurred Cost'

plot.line('x', 'y', legend='category', source=plan1['source'], line_width=3, line_alpha=0.6, muted_alpha=0.2)
plot.line('x', 'y', legend='category', source=plan2['source'], line_width=3, line_alpha=0.6, muted_alpha=0.2, line_color="orange", muted_color="orange")

plot.legend.location = "top_left"
plot.legend.click_policy="mute"

callback1 = CustomJS(args=plan1,code=callback_code)
callback2 = CustomJS(args=plan2,code=callback_code)

plan1['label'].js_on_change('value', callback1)
plan1['premium'].js_on_change('value', callback1)
plan1['contribution'].js_on_change('value', callback1)
plan1['deductible'].js_on_change('value', callback1)
plan1['coinsurance'].js_on_change('value', callback1)
plan1['maxOutOfPocket'].js_on_change('value', callback1)

plan2['label'].js_on_change('value', callback2)
plan2['premium'].js_on_change('value', callback2)
plan2['contribution'].js_on_change('value', callback2)
plan2['deductible'].js_on_change('value', callback2)
plan2['coinsurance'].js_on_change('value', callback2)
plan2['maxOutOfPocket'].js_on_change('value', callback2)

In [3]:
layout = row(
    plot,
    column(plan1['label'],plan1['premium'],plan1['contribution'],plan1['deductible'],plan1['coinsurance'],plan1['maxOutOfPocket']),
    column(plan2['label'],plan2['premium'],plan2['contribution'],plan2['deductible'],plan2['coinsurance'],plan2['maxOutOfPocket']),
)

output_file("index.html", title="Insurance Plan Comparison")

show(layout)